# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [54]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [55]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_1 import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [56]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [57]:
data['year1ChangePercent']

-0.21980382774404966

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [58]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [59]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                ],
                index=my_columns,
            ),
            ignore_index=True,
        )


final_dataframe


/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,154.500,-0.007692,N/A
1,AAL,12.900,-0.247690,N/A
2,AAP,148.293,-0.386695,N/A
3,AAPL,136.320,-0.219117,N/A
4,ABBV,168.030,0.294501,N/A
...,...,...,...,...
95,CINF,104.320,-0.075737,N/A
96,CL,81.670,-0.035521,N/A
97,CLX,150.300,-0.145013,N/A
98,CMA,66.260,-0.220170,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [60]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,42,APA,44.340,0.829386,N/A
1,77,CAH,81.550,0.657451,N/A
2,94,CI,345.011,0.522429,N/A
3,89,CF,97.190,0.507492,N/A
4,11,ADM,95.390,0.454285,N/A
5,5,ABC,169.240,0.353928,N/A
6,4,ABBV,168.030,0.294501,N/A
7,39,ANTM,489.250,0.292205,N/A
8,17,AFL,70.860,0.250873,N/A
9,48,ATVI,77.040,0.250428,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [61]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

10000000


In [62]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(1):#, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
42,APA,44.340,0.829386,N/A
77,CAH,81.550,0.657451,N/A
94,CI,345.011,0.522429,N/A
89,CF,97.190,0.507492,N/A
11,ADM,95.390,0.454285,N/A
5,ABC,169.240,0.353928,N/A
4,ABBV,168.030,0.294501,N/A
39,ANTM,489.250,0.292205,N/A
17,AFL,70.860,0.250873,N/A
48,ATVI,77.040,0.250428,N/A


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [63]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile",
    "HQM Score",
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings[:1]:
    #     print(symbol_strings)
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    "N/A",
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month6ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month3ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month1ChangePercent"],
                    "N/A",
                    "N/A",
                ],
                index=hqm_columns,
            ),
            ignore_index=True,
        )

hqm_dataframe.columns


/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/1599880919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/1599880919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/1599880919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/1599880919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [64]:
time_periods = ["One-Year", "Six-Month", "Three-Month", "One-Month"]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f"{time_period} Return Percentile"] = (
            stats.percentileofscore(
                hqm_dataframe[f"{time_period} Price Return"],
                hqm_dataframe.loc[row, f"{time_period} Price Return"],
            )
            / 100
        )

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f"{time_period} Return Percentile"])

# Print the entire DataFrame
hqm_dataframe


0     0.61
1     0.25
2     0.12
3     0.34
4     0.94
      ... 
95    0.51
96    0.57
97    0.42
98    0.32
99     0.2
Name: One-Year Return Percentile, Length: 100, dtype: object
0     0.91
1     0.26
2      0.1
3     0.36
4     0.72
      ... 
95    0.17
96    0.41
97    0.75
98    0.11
99    0.16
Name: Six-Month Return Percentile, Length: 100, dtype: object
0     0.87
1     0.22
2     0.13
3     0.18
4      0.8
      ... 
95     0.6
96    0.53
97    0.48
98    0.09
99    0.44
Name: Three-Month Return Percentile, Length: 100, dtype: object
0     0.76
1     0.11
2     0.35
3     0.08
4     0.88
      ... 
95    0.37
96    0.74
97    0.54
98    0.15
99    0.64
Name: One-Month Return Percentile, Length: 100, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,148.500,N/A,-0.007719,0.61,0.336773,0.91,0.158558,0.87,0.021820,0.76,N/A
1,AAL,12.690,N/A,-0.251624,0.25,-0.010316,0.26,-0.068790,0.22,-0.090103,0.11,N/A
2,AAP,146.586,N/A,-0.378519,0.12,-0.155451,0.1,-0.154413,0.13,-0.040250,0.35,N/A
3,AAPL,138.187,N/A,-0.210108,0.34,0.026183,0.36,-0.110453,0.18,-0.111044,0.08,N/A
4,ABBV,165.711,N/A,0.295273,0.94,0.187587,0.72,0.128499,0.8,0.036152,0.88,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,104.320,N/A,-0.075955,0.51,-0.092855,0.17,0.058216,0.6,-0.037619,0.37,N/A
96,CL,79.130,N/A,-0.036006,0.57,0.063982,0.41,0.034671,0.53,0.017739,0.74,N/A
97,CLX,152.610,N/A,-0.144884,0.42,0.203536,0.75,0.029619,0.48,-0.015525,0.54,N/A
98,CMA,65.610,N/A,-0.212940,0.32,-0.134004,0.11,-0.181373,0.09,-0.080840,0.15,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [65]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,148.500,N/A,-0.007719,0.61,0.336773,0.91,0.158558,0.87,0.021820,0.76,0.7875
1,AAL,12.690,N/A,-0.251624,0.25,-0.010316,0.26,-0.068790,0.22,-0.090103,0.11,0.21
2,AAP,146.586,N/A,-0.378519,0.12,-0.155451,0.1,-0.154413,0.13,-0.040250,0.35,0.175
3,AAPL,138.187,N/A,-0.210108,0.34,0.026183,0.36,-0.110453,0.18,-0.111044,0.08,0.24
4,ABBV,165.711,N/A,0.295273,0.94,0.187587,0.72,0.128499,0.8,0.036152,0.88,0.835
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,104.320,N/A,-0.075955,0.51,-0.092855,0.17,0.058216,0.6,-0.037619,0.37,0.4125
96,CL,79.130,N/A,-0.036006,0.57,0.063982,0.41,0.034671,0.53,0.017739,0.74,0.5625
97,CLX,152.610,N/A,-0.144884,0.42,0.203536,0.75,0.029619,0.48,-0.015525,0.54,0.5475
98,CMA,65.610,N/A,-0.212940,0.32,-0.134004,0.11,-0.181373,0.09,-0.080840,0.15,0.1675


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [66]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,77,CAH,82.610,N/A,0.659663,0.99,0.603290,0.99,0.202727,0.93,0.021411,0.75,0.915
1,43,APD,321.130,N/A,0.088070,0.73,0.372675,0.96,0.289954,0.96,0.068259,0.96,0.9025
2,94,CI,343.490,N/A,0.510081,0.98,0.349896,0.93,0.133665,0.83,0.029980,0.85,0.8975
3,6,ABMD,395.450,N/A,0.216572,0.87,0.679505,1.0,0.479439,1.0,0.010105,0.72,0.8975
4,5,ABC,171.680,N/A,0.358385,0.95,0.206384,0.77,0.202890,0.94,0.029651,0.84,0.875
5,18,AIG,63.221,N/A,0.186902,0.82,0.233892,0.81,0.172241,0.9,0.052177,0.93,0.865
6,55,BA,189.384,N/A,-0.041169,0.56,0.357362,0.94,0.292479,0.97,0.062913,0.95,0.855
7,76,CAG,38.512,N/A,0.141678,0.78,0.235497,0.83,0.140395,0.84,0.058655,0.94,0.8475
8,16,AES,28.840,N/A,0.216387,0.86,0.514368,0.98,0.104107,0.75,0.024757,0.79,0.845
9,4,ABBV,165.711,N/A,0.295273,0.94,0.187587,0.72,0.128499,0.8,0.036152,0.88,0.835


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [67]:
portfolio_input()

In [69]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,77,CAH,82.610,242,0.659663,0.99,0.603290,0.99,0.202727,0.93,0.021411,0.75,0.915
1,43,APD,321.130,62,0.088070,0.73,0.372675,0.96,0.289954,0.96,0.068259,0.96,0.9025
2,94,CI,343.490,58,0.510081,0.98,0.349896,0.93,0.133665,0.83,0.029980,0.85,0.8975
3,6,ABMD,395.450,50,0.216572,0.87,0.679505,1.0,0.479439,1.0,0.010105,0.72,0.8975
4,5,ABC,171.680,116,0.358385,0.95,0.206384,0.77,0.202890,0.94,0.029651,0.84,0.875
5,18,AIG,63.221,316,0.186902,0.82,0.233892,0.81,0.172241,0.9,0.052177,0.93,0.865
6,55,BA,189.384,105,-0.041169,0.56,0.357362,0.94,0.292479,0.97,0.062913,0.95,0.855
7,76,CAG,38.512,519,0.141678,0.78,0.235497,0.83,0.140395,0.84,0.058655,0.94,0.8475
8,16,AES,28.840,693,0.216387,0.86,0.514368,0.98,0.104107,0.75,0.024757,0.79,0.845
9,4,ABBV,165.711,120,0.295273,0.94,0.187587,0.72,0.128499,0.8,0.036152,0.88,0.835


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [71]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [72]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [73]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["Price", dollar_template],
    "C": ["Number of Shares to Buy", integer_template],
    "D": ["One-Year Price Return", percent_template],
    "E": ["One-Year Return Percentile", percent_template],
    "F": ["Six-Month Price Return", percent_template],
    "G": ["Six-Month Return Percentile", percent_template],
    "H": ["Three-Month Price Return", percent_template],
    "I": ["Three-Month Return Percentile", percent_template],
    "J": ["One-Month Price Return", percent_template],
    "K": ["One-Month Return Percentile", percent_template],
    "L": ["HQM Score", integer_template],
}

for column in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(
        f"{column}:{column}", 20, column_formats[column][1]
    )
    writer.sheets["Momentum Strategy"].write(
        f"{column}1", column_formats[column][0], string_template
    )


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [74]:
writer.save()

/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_10212/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
